In [2]:
####################################Imports and Initial Feed Rip###################################################



#Requires PySpark, installation information can be found here: https://www.dataquest.io/blog/installing-pyspark/
#import PySpark as ps 
import feedparser as fp, pandas as pd, datetime, time
from bs4 import BeautifulSoup 
from functools import reduce

#Get the News Feed and Slice it to only include the news items
feed = fp.parse('http://feeds.huffingtonpost.com/c/35496/f/677097/index.rss')
feed_items = feed['items']

###################################################################################################################

In [3]:
#################################Create the Cleaned News Items DataFrame###########################################

#Import the raw data into a DataFrame
raw_news_dataframe = pd.DataFrame(feed_items)

#Clean the DataFrame
def dataframe_cleaner(dataframe): 
    being_cleaned_df = pd.DataFrame(raw_news_dataframe[['author','id','title','summary']])
    
    for i in range(0,len(being_cleaned_df['summary'])):
        soup = BeautifulSoup(being_cleaned_df['summary'][i],'lxml')
        being_cleaned_df['summary'][i] = soup.get_text()
    
    return being_cleaned_df
    
#Create the cleaned Dataframe    
news_df = dataframe_cleaner(raw_news_dataframe)
news_df.head(1)

###################################################################################################################

,author,id,title,summary
0,Nick Visser,http://www.huffingtonpost.com/2016/03/29/beam-...,NASA Test Could Pave The Way For Future Space ...,NASA is preparing to launch an expandable c...


In [7]:
#Load in the Sentiment DataFrame
sentiment_df = pd.read_csv('sentiment_DataFrame.csv')
sentiment_df = sentiment_df.ix[:,'type':]
sentiment_df.head(1)

,type,word,position,stemmed,polarity
0,0.5,abandoned,adj,n,-1


In [4]:
##########################Perform Sentiment Analysis on the Title and Body of the Articles#########################
def title_analysis(title):
    tokenized_title = dict.fromkeys(title.split(' '))
    
    for key in tokenized_title:
        tokenized_title[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    sentiment = sum([item for sublist in list(tokenized_title.values()) for item in sublist])
    return sentiment

def summary_analysis(summary):
    tokenized_summary = dict.fromkeys(summary.split(' '))
    
    for key in tokenized_summary:
        tokenized_summary[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    sentiment = sum([item for sublist in list(tokenized_summary.values()) for item in sublist])
    return sentiment

def sentiment_analysis(dataframe):
    analyzing_df = dataframe
    analyzing_df['title_sentiment'] = list(map(lambda x: title_analysis(x),analyzing_df['title']))
    analyzing_df['summary_sentiment'] = list(map(lambda x: summary_analysis(x),analyzing_df['summary']))
    return analyzing_df

start_time = time.time()
analyzed_articles_df = sentiment_analysis(news_df)
elapsed_time = time.time() - start_time
print('Elapsed Time: ' + str(elapsed_time))
analyzed_articles_df.head()

###################################################################################################################

Elapsed Time: 521.7441730499268


,author,id,title,summary,title_sentiment,summary_sentiment
0,James Cave,http://www.huffingtonpost.com/2016/03/29/umbre...,Umbrellas With Lifetime Warranties Will Weathe...,"When you think about it, umbrella technology r...",2.0,10.0
1,Jessica Samakow,http://www.huffingtonpost.com/2016/03/28/babes...,Dudes Launch 'Babes For Trump' Instagram To El...,Women have strong political convictions; wome...,0.5,26.5
2,Natalia Brzezinski,http://www.huffingtonpost.com/natalia-lopatniu...,A Brilliant Mind: Founding Tech Companies with...,There's wide global awareness today that we ne...,2.5,58.5
3,Casey Williams,http://www.huffingtonpost.com/2016/03/25/low-i...,"For Poor Americans, Getting Online Is About Mo...",Internet access could soon get a whole lot ch...,0.5,5.5
4,Charles Edge,http://www.huffingtonpost.com/charles-edge/the...,The Importance Of Having A Good Content Strate...,Search engine optimization (SEO) involves stra...,2.5,36.0
